In [10]:
import pandas as pd 
import numpy as np
import warnings
import time
import math
# import cvxpy as cp
from ortools.linear_solver import pywraplp
warnings.filterwarnings('ignore')

### Ver 3.4

In [11]:
def get_po_data(df_po,Thickness,Spec,stock_name): #
    data= df_po
    con1 = data['Need_Cut'] < 0
    con2 = data['Stock_Name'] == stock_name
    con3 = data['Thickness'] == Thickness
    # con4 = data['Cutting_Note'].isna()
    con5 = data['Spec_Name'] == Spec
    # con6 = data['Customer'] == Customer
    # con7 = data['Length'] == Length
    
    data[['Thickness','Need_Cut']] = data[['Thickness','Need_Cut']].astype('float64')
    data['Need_Cut']= round(data['Need_Cut'])

    tmp= data[con2 & con3 & con5].sort_values(by= ['Need_Cut', 'Width'], ascending= [True, True]).reset_index(drop= True) #& con6
    tmp['Order_ID'] = tmp.index
    return tmp

def get_stock_data(df_stock,Thickness,Spec,sort = [True]):
    st_con1= df_stock['Spec_Name'] == Spec
    st_con2= df_stock['Thickness'] == Thickness
    # st_con3= df_stock['Remark'].isin(['Ok',np.nan])
    # st_con4= df_stock['Cus_Number'].isna()
    # st_con5= df_stock['Width'] == Width
    tmp_stock = df_stock[st_con1 & st_con2 ] #& st_con5
    tmp_st = tmp_stock[['Inventory_ID','Spec_Name','Thickness','Width','Length','Weigth']].sort_values(by= ['Weigth'], ascending= sort).reset_index(drop=True)
    # tmp_stock['Receipt_Date'] = tmp_stock['Receipt_Date'].astype('datetime64[ns]')
    # tmp_stock['HTV_Note'] = tmp_stock['HTV_Note'].replace(0, np.nan)
    # tmp_st = tmp_stock[tmp_stock['HTV_Note'].isna()].sort_values(by= ['Weigth','Receipt_Date'], ascending= [False, True]).reset_index(drop=True)[['Inventory_ID','Spec_Name','Thickness','Width','Length','Quantity','Weigth','Receipt_Date']]
    # tmp_st = tmp_stock.sort_values(by= ['Weigth','Receipt_Date'], ascending= [False, True]).reset_index(drop=True)[['Inventory_ID','Spec_Name','Thickness','Width','Length','Quantity','Weigth','Receipt_Date']]
    return tmp_st

def process_data(df,df_stock):
    df['Mother_Weight'] = df_stock['Weigth'][0]
    df['Mother_Width']= df_stock['Width'][0]
    df['Inventory_ID'] = df_stock['Inventory_ID'][0]
    df['Expected_Qt']=  df.apply(lambda df: 0 if df['Need_Cut']>=0 else round(abs(df['Remain_Weight']/(df['Width']*df_stock['Weigth'][0]/df_stock['Width'][0])) + 0.5), axis=1) #+ 0.5
    
    df['Trim_Loss'] = df_stock['Width'][0]
    df['Current_Weight'] = 0  
    df['Max_Quant']= df.apply(lambda df: 0 if df['Need_Cut']>=0 else round((df['Max_Weight']*df_stock['Width'][0])/(df_stock['Weigth'][0]*df['Width']) + 0.5 ), axis= 1) #+ 0.5        
    
    df_stock = df_stock.drop(index=0).reset_index(drop= True)
    df = df[['Order_ID','Customer', 'Maker', 'Spec_Name', 'Thickness', 'Width', 'Length', 
             'Need_Cut', 'Inventory_ID','Mother_Weight', 'Mother_Width','Max_Weight', 'Expected_Qt',
             'Num_Split','Max_Quant','Total_Quant','Trim_Loss','Current_Weight','Remain_Weight']]
    # df_nc = df[df.Need_Cut >= 0].reset_index(drop= True)
    df = df[df.Need_Cut <0].reset_index(drop= True)
    return df, df_stock

def create_data_model(df,Width,min_loss,max_loss):
    """Stores the data for the problem."""
    data = {}
    data['constraint_coeffs'] = [
        list(df['Width'].values),
        # list(-df['Width'].values),
        ]
    data['bounds'] = [Width - min_loss] # Width - min_loss, 
    data['obj_coeffs'] = list(df['Width'].values) 
    data['num_vars'] = len(df)
    data['num_constraints'] = len(data['constraint_coeffs'])
    return data

def optimize_model_v4(df,Width,min_loss,max_loss, max_quant = False):     
    data = create_data_model(df,Width,min_loss,max_loss)
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP') #GLOP  SCIP  PDLP Clp  GLPK  CBC  XPRESS SAT

    if not solver:
        return

    # infinity = solver.infinity()
    x = {}
    if max_quant == False:
        for j in range(data['num_vars']):
            x[j] = solver.IntVar(0, solver.infinity(), 'x[%i]' % j) #int(tmp_['Max_Quant'][j])
    else:
        for j in range(data['num_vars']):
            x[j] = solver.IntVar(0, int(df['Max_Quant'][j]), 'x[%i]' % j) #int(tmp_['Max_Quant'][j])
    # print('Number of variables =', solver.NumVariables())

    for i in range(data['num_constraints']):
     constraint_expr = \
    [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
     solver.Add(sum(constraint_expr) <= data['bounds'][i])
    
    obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
    solver.Minimize((Width - min_loss)  - solver.Sum(obj_expr)) #(Width - min_loss)
    
    status = solver.Solve()
    # print(solver.Objective().Gap())
    # print(status)
    if status == pywraplp.Solver.OPTIMAL:
        sol_values = []
        # print('Objective value =', solver.Objective().Value())
        for j in range(data['num_vars']):
            sol_values.append(x[j].solution_value())
            # print(x[j].name(), ' = ', x[j].solution_value())
        # print(solver.Objective().BestBound())
        # print('Problem solved in %f milliseconds' % solver.wall_time())
        # print('Problem solved in %d iterations' % solver.iterations())
        # print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
        # return sol_values , Width - solver.Objective().Value(), solver.wall_time()
    else:
        print('The problem does not have an optimal solution.')
        # return 0 , Width, solver.wall_time()
    # return sol_values , Width - solver.Objective().Value(), solver.wall_time()
    trim_loss = min_loss + solver.Objective().Value() #min_loss + 
    # trim_loss = solver.Objective().Value()
    # trim_loss = max_loss - solver.Objective().Value()
    
    return sol_values ,trim_loss, solver.wall_time()

In [12]:
def sel_extra_coil(df, df_1):
    results = 0
    # df_1.Expected_Qt = 2
    df_1 = df_1[~df_1.Order_ID.isin(df.Order_ID.unique())]
    if len(df_1) >0:
        for i in ['Inventory_ID', 'Mother_Weight', 'Mother_Width']:
            df_1[i] = df[i][0]
        df_1[['Expected_Qt', 'Trim_Loss', 'Current_Weight','Run_Time(ms)']] = 0
        df_1['Max_Quant'] = 2
        # df_1[['Inventory_ID', 'Mother_Weight', 'Mother_Width', 'Expected_Qt', 'Max_Quant', 'Trim_Loss', 'Current_Weight']] = 0
        df_1 = df_1[list(df.columns)]

        df_tmp = pd.DataFrame(columns= df.columns)
        # data = pd.DataFrame(columns= df.columns)
        for i in range(len(df_1)):
            data= pd.concat([df,df_1[i:i+1]]).reset_index(drop=True)
            try:
                # print(df_1[i:i+1].Order_ID.unique())
                Num_Split,Trim_Loss,Run_Time = optimize_model_v4(data,Width,min_loss,max_loss, True)
            except:
                Trim_Loss = df['Mother_Width'][0]
            if i == 0:
                results = Trim_Loss
                df_tmp = data
            if Trim_Loss < results:
                results = Trim_Loss
                df_tmp = data
        return df_tmp
    else:
        return df

In [13]:
def main_v1(df,df_stock,process_type = 'Default',percent = 1,max_w_1= True,max_w_2=True):
    start = time.time()
    df['Num_Split'] = 0
    if process_type == 'Default':
        df['Max_Weight'] = df.apply(lambda df : 0 if df['Need_Cut'] >= 0 else abs(df['Need_Cut']),1)
    elif process_type == 'Auto':
        df['Max_Weight'] = abs(df['Need_Cut'])*round(percent,2)
    else:
        df['Max_Weight'] = 0
        for or_id, max_weight in process_type: 
            df.at[df[df.Order_ID == or_id].index[0], 'Max_Weight'] = max_weight

    df['Total_Quant'] = 0
    df['Remain_Weight'] = df['Need_Cut']

    df_nc = df.copy()
    df_nc[['Inventory_ID', 'Mother_Weight', 'Mother_Width', 'Expected_Qt', 'Max_Quant', 'Trim_Loss', 'Current_Weight']] = 0
    df,df_stock = process_data(df, df_stock)
    final_output = pd.DataFrame(columns= df.columns)
    
    Width= df['Mother_Width'][0]
    Thickness= df['Thickness'][0]

    if Thickness <= 3 and Thickness > 1:
        min_loss = 8
    elif Thickness == 1:
        min_loss = 5.5
    else:
        min_loss = 3*Thickness      
    max_loss= int(0.04*Width)   
    try:
        i=0
        while len(df) > 0:
            print(f'Lần cắt thứ {i+1}')
            Num_Split,Trim_Loss,Run_Time = optimize_model_v4(df,Width,min_loss,max_loss, max_w_1)
            
            if round(Trim_Loss,2) > round(min_loss,2):    #max_loss
                df_op = df.copy()
                df_op_final =  df.copy()
                df_op_final['Num_Split'],df_op_final['Trim_Loss'],df_op_final['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
                cur_loss = Trim_Loss #len(df) + Trim_Loss
                for lop in range(len(df_nc)):
                    # print(cur_loss)
                    df_op = sel_extra_coil(df_op, df_nc)
                    try:
                        Num_Split,Trim_Loss,Run_Time = optimize_model_v4(df_op,Width,min_loss,max_loss, max_w_2)
                        df_op['Num_Split'],df_op['Trim_Loss'],df_op['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
                        df_check = df_op[df_op.Num_Split < df_op.Expected_Qt].reset_index(drop=True)
                        if len(df_check) == 0 and Trim_Loss <= max_loss:
                            check_point = Trim_Loss/10
                        else:
                            check_point = Trim_Loss     #len(df_check) + Trim_Loss
                        # check_point = len(df_check) + Trim_Loss
                        print('Optimizing Trim Loss ...... ',Trim_Loss)
                        # if int(check_point) <= int(min_loss):
                        #     df_op_final = df_op
                        #     break
                        if len(df_check) ==0 and int(Trim_Loss) == min_loss:  #<= int(max_loss/2)
                            df_op_final = df_op
                            break
                        if check_point < cur_loss:
                            cur_loss = check_point
                            df_op_final = df_op
                    except:
                        pass
                df = df_op_final    
            else:
                df['Num_Split'],df['Trim_Loss'],df['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
                
            df['Total_Quant'] += df['Num_Split']
            df['Current_Weight'] = (df['Num_Split']*df.Width*df['Mother_Weight'])/df['Mother_Width']
            df['Remain_Weight'] += df['Current_Weight']
            df['Max_Weight'] = df.apply(lambda df: df['Max_Weight'] - df['Current_Weight'] if df['Max_Weight'] != 0 else df['Max_Weight'],1)
            df['Trim_Loss_Weight'] = df.apply(lambda df: (df['Trim_Loss']*df['Mother_Weight'])/df['Mother_Width'],1)

            final_output= pd.concat([final_output,df[df.Num_Split > 0]]).reset_index(drop=True)
            df = df[df.Num_Split < df.Expected_Qt].reset_index(drop=True)

            i += 1
            # print(final_output)
            if len(df) != 0 :
                try:
                    df,df_stock = process_data(df, df_stock)
                    print(f'Số Coil còn lại: {len(df)}')
                except:
                    print(f'Trong kho có {i} MC ... Số MC này không đủ để cắt !!!! ')
                    print(f'Số Coil còn lại: {len(df)}')
                    break
            if i > 10:
                break
    except:
        print('Ăn lol rồi !!!')
        i=0
        while len(df) > 0:
            print(f'Lần cắt thứ {i+1}')
            Num_Split,Trim_Loss,Run_Time = optimize_model_v4(df,Width,min_loss,max_loss, max_w_1)
            
            if Trim_Loss > max_loss:    #max_loss
                df_op = df.copy()
                df_op_final =  df.copy()
                cur_loss = len(df) + Trim_Loss
                for lop in range(len(df_nc)):
                    # print(cur_loss)
                    df_op = sel_extra_coil(df_op, df_nc)
                    try:
                        Num_Split,Trim_Loss,Run_Time = optimize_model_v4(df_op,Width,min_loss,max_loss, max_w_2)
                        df_op['Num_Split'],df_op['Trim_Loss'],df_op['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
                        df_check = df_op[df_op.Num_Split < df_op.Expected_Qt].reset_index(drop=True)
                        if len(df_check) == 0 and Trim_Loss <= max_loss:
                            check_point = Trim_Loss/10
                        else:
                            check_point = len(df_check) + Trim_Loss
                        # check_point = len(df_check) + Trim_Loss
                        print('Optimizing Trim Loss ...... ',Trim_Loss)
                        # if int(check_point) <= int(min_loss):
                        #     df_op_final = df_op
                        #     break
                        if len(df_check) ==0 and int(Trim_Loss) == min_loss:  #<= int(max_loss/2)
                            df_op_final = df_op
                            break
                        if check_point < cur_loss:
                            cur_loss = check_point
                            df_op_final = df_op
                    except:
                        pass
                df = df_op_final    
            else:
                df['Num_Split'],df['Trim_Loss'],df['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
                
            df['Total_Quant'] += df['Num_Split']
            df['Current_Weight'] = (df['Num_Split']*df.Width*df['Mother_Weight'])/df['Mother_Width']
            df['Remain_Weight'] += df['Current_Weight']
            df['Max_Weight'] = df.apply(lambda df: df['Max_Weight'] - df['Current_Weight'] if df['Max_Weight'] != 0 else df['Max_Weight'],1)
            df['Trim_Loss_Weight'] = df.apply(lambda df: (df['Trim_Loss']*df['Mother_Weight'])/df['Mother_Width'],1)

            final_output= pd.concat([final_output,df[df.Num_Split > 0]]).reset_index(drop=True)
            df = df[df.Num_Split < df.Expected_Qt].reset_index(drop=True)

            i += 1
            # print(final_output)
            if len(df) != 0 :
                try:
                    df,df_stock = process_data(df, df_stock)
                    print(f'Số Coil còn lại: {len(df)}')
                except:
                    print(f'Trong kho có {i} MC ... Số MC này không đủ để cắt !!!! ')
                    print(f'Số Coil còn lại: {len(df)}')
                    break
            if i > 10:
                break
            
    final_output['Total_MC']= i
    final_output['Total_Remain']= sum(final_output['Remain_Weight'][final_output['Remain_Weight'] >=0 ])
    final_output['Total_Trim_Loss_Weight'] = float(final_output.groupby(['Inventory_ID'])['Trim_Loss_Weight'].unique().reset_index().Trim_Loss_Weight.sum())
    final_output['Total_Run_Time(s)'] = time.time() - start
    #float(final_output.groupby(['Inventory_ID'])['Run_Time(ms)'].unique().reset_index()['Run_Time(ms)'].sum())
    return final_output,df

In [14]:
def main_v2(df,df_stock,process_type = 'Default',percent = 1,max_w_1= True,max_w_2=True):
    start = time.time()
    df['Num_Split'] = 0
    if process_type == 'Default':
        df['Max_Weight'] = df.apply(lambda df : 0 if df['Need_Cut'] >= 0 else abs(df['Need_Cut']),1)

    df['Total_Quant'] = 0
    df['Remain_Weight'] = df['Need_Cut']

    df_nc = df.copy()
    df_nc[['Inventory_ID', 'Mother_Weight', 'Mother_Width', 'Expected_Qt', 'Max_Quant', 'Trim_Loss', 'Current_Weight']] = 0

    df,df_stock = process_data(df, df_stock)
    final_output = pd.DataFrame(columns= df.columns)
    df_check = df.copy()

    Width= df['Mother_Width'][0]
    Thickness= df['Thickness'][0]
    df_nc = df_nc[df.columns]

    if Thickness <= 3 and Thickness > 1:
        min_loss = 8     #8
    elif Thickness == 1:
        min_loss = 5.5
    else:
        min_loss = 3*Thickness      
    max_loss= int(0.04*Width)   
    i=0
###################################################
    while len(df) > 0:
        print(f'Lần cắt thứ {i+1}')
        Num_Split,Trim_Loss,Run_Time = optimize_model_v4(df,Width,min_loss,max_loss, max_w_1)
        # print(Trim_Loss)

        if sum(df.Expected_Qt) == sum(Num_Split) and Trim_Loss < max_loss:
            df['Num_Split'],df['Trim_Loss'],df['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
            
        elif sum(Num_Split) == sum(df.Expected_Qt) and Trim_Loss >= max_loss:
            # print('2')
            df['Num_Split'],df['Trim_Loss'],df['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
            df_nc_new = df_nc.copy()
            Width_new = Trim_Loss
            Num_Split_new,Trim_Loss_new,Run_Time_new = optimize_model_v4(df_nc_new,Width_new,min_loss,max_loss, False)
            df_nc_new['Num_Split'],df_nc_new['Trim_Loss'],df_nc_new['Run_Time(ms)'] = Num_Split_new,Trim_Loss_new,Run_Time_new
            for cols in ['Inventory_ID', 'Mother_Weight', 'Mother_Width']:
                df_nc_new[cols] = df[cols][0]

            df = pd.concat([df, df_nc_new[df_nc_new.Num_Split > 0]])
            df['Trim_Loss'] = Trim_Loss_new
            df['Run_Time(ms)'] += Run_Time_new
            # print(Trim_Loss_new)
            
        # elif round(Trim_Loss,2) > round(min_loss,2):
        #     print('3')
        #     df['Num_Split'],df['Trim_Loss'],df['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
        #     df_nc_new = df_nc.copy()
        #     Width_new = Trim_Loss
        #     Num_Split_new,Trim_Loss_new,Run_Time_new = optimize_model_v4(df_nc_new,Width_new,min_loss,max_loss, False)
        #     df_nc_new['Num_Split'],df_nc_new['Trim_Loss'],df_nc_new['Run_Time(ms)'] = Num_Split_new,Trim_Loss_new,Run_Time_new
        #     for cols in ['Inventory_ID', 'Mother_Weight', 'Mother_Width']:
        #         df_nc_new[cols] = df[cols][0]

        #     df = pd.concat([df, df_nc_new[df_nc_new.Num_Split > 0]])
        #     df['Trim_Loss'] = Trim_Loss_new
        #     df['Run_Time(ms)'] += Run_Time_new
        #     print(Trim_Loss_new)
                
        elif round(Trim_Loss,2) > round(min_loss,2) : 
            df_op = df.copy()
            df_op_final =  df.copy()
            df_op_final['Num_Split'],df_op_final['Trim_Loss'],df_op_final['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
            cur_loss = len(df) + Trim_Loss
            for lop in range(len(df_nc)):
                # print(cur_loss)
                df_op = sel_extra_coil(df_op, df_nc)
                try:
                    Num_Split,Trim_Loss,Run_Time = optimize_model_v4(df_op,Width,min_loss,max_loss, max_w_2)
                    df_op['Num_Split'],df_op['Trim_Loss'],df_op['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
                    df_check = df_op[df_op.Num_Split < df_op.Expected_Qt].reset_index(drop=True)
                    if len(df_check) == 0 and Trim_Loss <= max_loss:
                        check_point = Trim_Loss/10
                    else:
                        check_point = len(df_check) + Trim_Loss
                    print('Optimizing Trim Loss ...... ',Trim_Loss)

                    if len(df_check) ==0 and int(Trim_Loss) == min_loss:  #<= int(max_loss/2)
                        df_op_final = df_op
                        break
                    if check_point < cur_loss:
                        cur_loss = check_point
                        df_op_final = df_op
                except:
                    pass
            df = df_op_final  

        else:
            # print(4)
            df['Num_Split'],df['Trim_Loss'],df['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time

        df['Total_Quant'] += df['Num_Split']
        df['Current_Weight'] = (df['Num_Split']*df.Width*df['Mother_Weight'])/df['Mother_Width']
        df['Remain_Weight'] += df['Current_Weight']
        df['Max_Weight'] = df.apply(lambda df: df['Max_Weight'] - df['Current_Weight'] if df['Max_Weight'] != 0 else df['Max_Weight'],1)
        df['Trim_Loss_Weight'] = df.apply(lambda df: (df['Trim_Loss']*df['Mother_Weight'])/df['Mother_Width'],1)

        final_output= pd.concat([final_output,df[df.Num_Split > 0]]).reset_index(drop=True)
        df = df[df.Num_Split < df.Expected_Qt].reset_index(drop=True)

        i += 1
        # print(final_output) 
        if len(df) != 0 :
            try:
                df,df_stock = process_data(df, df_stock)
                print(f'Số Coil còn lại: {len(df)}')
            except:
                print(f'Trong kho có {i} MC ... Số MC này không đủ để cắt !!!! ')
                print(f'Số Coil còn lại: {len(df)}')
                break
        if i > 15:
            break

        print('So coil con lai: ', len(df))
###################################################

    final_output['Total_MC']= i
    final_output['Total_Remain']= sum(final_output['Remain_Weight'][final_output['Remain_Weight'] >=0 ])
    final_output['Total_Trim_Loss_Weight'] = float(final_output.groupby(['Inventory_ID'])['Trim_Loss_Weight'].unique().reset_index().Trim_Loss_Weight.sum())
    # final_output['Total_Run_Time(s)'] = time.time() - start
    #float(final_output.groupby(['Inventory_ID'])['Run_Time(ms)'].unique().reset_index()['Run_Time(ms)'].sum())
    return final_output,df

In [15]:
def main_v3(df,df_stock,process_type = 'Default',percent = 1,max_w_1= True,max_w_2=True):
    start = time.time()
    df['Num_Split'] = 0
    if process_type == 'Default':
        df['Max_Weight'] = df.apply(lambda df : 0 if df['Need_Cut'] >= 0 else abs(df['Need_Cut']),1)

    df['Total_Quant'] = 0
    df['Remain_Weight'] = df['Need_Cut']

    df_nc = df.copy()
    df_nc[['Inventory_ID', 'Mother_Weight', 'Mother_Width', 'Expected_Qt', 'Max_Quant', 'Trim_Loss', 'Current_Weight']] = 0

    df,df_stock = process_data(df, df_stock)
    final_output = pd.DataFrame(columns= df.columns)
    df_check = df.copy()

    Width= df['Mother_Width'][0]
    Thickness= df['Thickness'][0]
    df_nc = df_nc[df.columns]

    if Thickness <= 3 and Thickness > 1:
        min_loss = 8     #8
    elif Thickness == 1:
        min_loss = 5.5
    else:
        min_loss = 3*Thickness      
    max_loss= int(0.04*Width)   
    i=0
###################################################
    while len(df) > 0:
        print(f'Lần cắt thứ {i+1}')
        Num_Split,Trim_Loss,Run_Time = optimize_model_v4(df,Width,min_loss,max_loss, max_w_1)
        # print(Trim_Loss)

        if sum(df.Expected_Qt) == sum(Num_Split) and Trim_Loss < max_loss:
            df['Num_Split'],df['Trim_Loss'],df['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
            
        elif sum(Num_Split) == sum(df.Expected_Qt) and Trim_Loss >= max_loss:
            # print('2')
            df['Num_Split'],df['Trim_Loss'],df['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
            df_nc_new = df_nc.copy()
            Width_new = Trim_Loss
            Num_Split_new,Trim_Loss_new,Run_Time_new = optimize_model_v4(df_nc_new,Width_new,min_loss,max_loss, False)
            df_nc_new['Num_Split'],df_nc_new['Trim_Loss'],df_nc_new['Run_Time(ms)'] = Num_Split_new,Trim_Loss_new,Run_Time_new
            for cols in ['Inventory_ID', 'Mother_Weight', 'Mother_Width']:
                df_nc_new[cols] = df[cols][0]

            df = pd.concat([df, df_nc_new[df_nc_new.Num_Split > 0]])
            df['Trim_Loss'] = Trim_Loss_new
            df['Run_Time(ms)'] += Run_Time_new
            # print(Trim_Loss_new)
            
        elif round(Trim_Loss,2) > round(min_loss,2):
            print('3')
            df['Num_Split'],df['Trim_Loss'],df['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
            df_nc_new = df_nc.copy()
            Width_new = Trim_Loss
            Num_Split_new,Trim_Loss_new,Run_Time_new = optimize_model_v4(df_nc_new,Width_new,min_loss,max_loss, False)
            df_nc_new['Num_Split'],df_nc_new['Trim_Loss'],df_nc_new['Run_Time(ms)'] = Num_Split_new,Trim_Loss_new,Run_Time_new
            for cols in ['Inventory_ID', 'Mother_Weight', 'Mother_Width']:
                df_nc_new[cols] = df[cols][0]

            df = pd.concat([df, df_nc_new[df_nc_new.Num_Split > 0]])
            df['Trim_Loss'] = Trim_Loss_new
            df['Run_Time(ms)'] += Run_Time_new
            # print(Trim_Loss_new)
                
        # elif round(Trim_Loss,2) > round(min_loss,2) : 
        #     df_op = df.copy()
        #     df_op_final =  df.copy()
        #     df_op_final['Num_Split'],df_op_final['Trim_Loss'],df_op_final['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
        #     cur_loss = len(df) + Trim_Loss
        #     for lop in range(len(df_nc)):
        #         # print(cur_loss)
        #         df_op = sel_extra_coil(df_op, df_nc)
        #         try:
        #             Num_Split,Trim_Loss,Run_Time = optimize_model_v4(df_op,Width,min_loss,max_loss, max_w_2)
        #             df_op['Num_Split'],df_op['Trim_Loss'],df_op['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time
        #             df_check = df_op[df_op.Num_Split < df_op.Expected_Qt].reset_index(drop=True)
        #             if len(df_check) == 0 and Trim_Loss <= max_loss:
        #                 check_point = Trim_Loss/10
        #             else:
        #                 check_point = len(df_check) + Trim_Loss
        #             print('Optimizing Trim Loss ...... ',Trim_Loss)

        #             if len(df_check) ==0 and int(Trim_Loss) == min_loss:  #<= int(max_loss/2)
        #                 df_op_final = df_op
        #                 break
        #             if check_point < cur_loss:
        #                 cur_loss = check_point
        #                 df_op_final = df_op
        #         except:
        #             pass
        #     df = df_op_final  

        else:
            # print(4)
            df['Num_Split'],df['Trim_Loss'],df['Run_Time(ms)'] = Num_Split,Trim_Loss,Run_Time

        df['Total_Quant'] += df['Num_Split']
        df['Current_Weight'] = (df['Num_Split']*df.Width*df['Mother_Weight'])/df['Mother_Width']
        df['Remain_Weight'] += df['Current_Weight']
        df['Max_Weight'] = df.apply(lambda df: df['Max_Weight'] - df['Current_Weight'] if df['Max_Weight'] != 0 else df['Max_Weight'],1)
        df['Trim_Loss_Weight'] = df.apply(lambda df: (df['Trim_Loss']*df['Mother_Weight'])/df['Mother_Width'],1)

        final_output= pd.concat([final_output,df[df.Num_Split > 0]]).reset_index(drop=True)
        df = df[df.Num_Split < df.Expected_Qt].reset_index(drop=True)

        i += 1
        # print(final_output) 
        if len(df) != 0 :
            try:
                df,df_stock = process_data(df, df_stock)
                print(f'Số Coil còn lại: {len(df)}')
            except:
                print(f'Trong kho có {i} MC ... Số MC này không đủ để cắt !!!! ')
                print(f'Số Coil còn lại: {len(df)}')
                break
        if i > 15:
            break

        print('So coil con lai: ', len(df))
###################################################

    final_output['Total_MC']= i
    final_output['Total_Remain']= sum(final_output['Remain_Weight'][final_output['Remain_Weight'] >=0 ])
    final_output['Total_Trim_Loss_Weight'] = float(final_output.groupby(['Inventory_ID'])['Trim_Loss_Weight'].unique().reset_index().Trim_Loss_Weight.sum())
    # final_output['Total_Run_Time(s)'] = time.time() - start
    #float(final_output.groupby(['Inventory_ID'])['Run_Time(ms)'].unique().reset_index()['Run_Time(ms)'].sum())
    return final_output,df

In [16]:
def score_(df_final_4,df_4,max_loss):
    status_4 = -1
    if len(df_final_4) > 0 and len(df_4) == 0 and max(df_final_4['Trim_Loss']) <= max_loss:  
        status_4 = 1
        print(f"""
        Total_MC: {df_final_4.Total_MC[0]}
        Total_Trim_Loss_Weight: {df_final_4.Total_Trim_Loss_Weight[0]}
        Total_Remain: {df_final_4.Total_Remain[0]}     
        Remain Order: {len(df_4)} 
        """)
        # score_4 = math.sqrt(math.pow(df_final_4.Total_MC[0],2) + 2*math.pow(df_final_4.Total_Trim_Loss_Weight[0],2) + math.pow(df_final_4.Total_Remain[0],2))
        # score_4 = math.sqrt(math.pow(df_final_4.Total_Trim_Loss_Weight[0],2) + math.pow(df_final_4.Total_Remain[0],2))
        score_4 = df_final_4.Total_Trim_Loss_Weight[0]
    else:
        score_4 = float('inf')
    return status_4,score_4, df_final_4.Total_Remain[0]

def output_table(df_po,df_st,Thickness,Spec,stock_name,Width = 1219):
    start = time.time()
    max_loss= int(0.04*Width) 
    po_data_raw = get_po_data(df_po,Thickness,Spec,stock_name)
    stock_data_raw = get_stock_data(df_st,Thickness,Spec)
    stock_data_raw4 = get_stock_data(df_st,Thickness,Spec,sort = [False])
    

    dfx = po_data_raw.reset_index(drop=True)
    df_stockx = stock_data_raw.reset_index(drop=True)
    df_stockx4 = stock_data_raw4.reset_index(drop=True)
    status= []
    score = []
    mc_ = []
    # -------------------------------------V2.1--------------------------------------------------
    df_final_1, df_1 = main_v2(dfx,df_stockx,process_type = 'Default',percent = 1,max_w_1= True,max_w_2=True)
    status_1,score_1,mc_1 = score_(df_final_1,df_1,max_loss)
    status.append(status_1)
    score.append(score_1)
    mc_.append(mc_1)
        
    df_final_2, df_2 = main_v2(dfx,df_stockx,process_type = 'Default',percent = 1,max_w_1= False,max_w_2=True)
    status_2,score_2,mc_2 = score_(df_final_2,df_2,max_loss)   
    status.append(status_2)
    score.append(score_2)
    mc_.append(mc_2)

    # -------------------------------------V2.2--------------------------------------------------
    df_final_3, df_3 = main_v2(dfx,df_stockx4,process_type = 'Default',percent = 1,max_w_1= True,max_w_2=True)
    status_3,score_3,mc_3 = score_(df_final_3,df_3,max_loss)
    status.append(status_3)
    score.append(score_3)
    mc_.append(mc_3)
    
    df_final_4, df_4 = main_v2(dfx,df_stockx4,process_type = 'Default',percent = 1,max_w_1= False,max_w_2=True)
    status_4,score_4,mc_4 = score_(df_final_4,df_4,max_loss)   
    status.append(status_4)
    score.append(score_4)
    mc_.append(mc_4)
    
    # -------------------------------------V3.1--------------------------------------------------
    df_final_5, df_5 = main_v3(dfx,df_stockx,process_type = 'Default',percent = 1,max_w_1= True)
    status_5,score_5,mc_5 = score_(df_final_5,df_5,max_loss)
    status.append(status_5)
    score.append(score_5)
    mc_.append(mc_5)
    
    df_final_6, df_6 = main_v3(dfx,df_stockx,process_type = 'Default',percent = 1,max_w_1= False)
    status_6,score_6,mc_6 = score_(df_final_6,df_6,max_loss) 
    status.append(status_6)
    score.append(score_6)
    mc_.append(mc_6)
    
    # -------------------------------------V3.2--------------------------------------------------   
    df_final_7, df_7 = main_v3(dfx,df_stockx4,process_type = 'Default',percent = 1,max_w_1= True)
    status_7,score_7,mc_7 = score_(df_final_7,df_7,max_loss)
    status.append(status_7)
    score.append(score_7)    
    mc_.append(mc_7)
    
    df_final_8, df_8 = main_v3(dfx,df_stockx4,process_type = 'Default',percent = 1,max_w_1= False)
    status_8,score_8,mc_8 = score_(df_final_8,df_8,max_loss)
    status.append(status_8)
    score.append(score_8)
    mc_.append(mc_8)

    # for i in range(len(status)):
    #     print(f'status {i+1} = {status[i]} and status_2 = {status_2} and status_2 = {status_3} and status_4 = {status_4}')
    #     print(f'score_1 = {score_1} and score_2 = {score_2} and score_3 = {score_3} and score_4 = {score_4}' )
    # # print('')
    
    print('Status: ', status)
    print('Score: ', score)
    print('Remain W: ',mc_)

    # min_ = min(score)
    # print(min_)
    chec_ = 0
    final_df = []
    inx = 0
    h = 0
    min_ = 0
    if 1 not in status:
        print('>>>>>>> Cant Find the Optimal Solution !!!! <<<<<<<')
        return
    
    for x,y,z in zip(status,score,mc_):
        if x == -1:
            pass
        else:
            if min_ == 0 and chec_ == 0:
                min_ = y
                chec_ = z
            elif y < min_:
                # if chec_ == 0:
                #     chec_ = z
                # elif z < chec_:
                inx = h
                min_ = y
        h += 1
                
    print('inx =', inx)
    
    if inx == 0:
        final_df = df_final_1
    elif inx == 1: 
        final_df = df_final_2
    elif inx == 2:
        final_df = df_final_3
    elif inx == 3:
        final_df = df_final_4
    elif inx == 4:
        final_df = df_final_5
    elif inx == 5:
        final_df = df_final_6
    elif inx == 6:
        final_df = df_final_7
    elif inx == 7:
        final_df = df_final_8 
    
    final_df['Total_Run_Time(s)'] = time.time() - start 
    # print('Total Run Time.....',time.time() - start)
    print('score = ',min_)
    return final_df

### Test Cases List:

1) 
Thickness = 2.6
Spec =  'JSH270C-PO'

2) 
Thickness = 1.6
Spec =  'JSH270C-PO'

3) 
Thickness = 1
Spec =  'SPCC-SD'   

4)
Thickness = 3.2
Spec =  'JSH440W-PO'   

5)
Thickness = 4
Spec =  'JSH270C-PO'  

6)
Thickness = 2.3
Spec =  'SS400-PO'  

7)
Thickness = 2.3
Spec =  'JSH270C-PO'  

In [17]:
stock_name = 'HSC'	
Thickness = 2.3
Spec =  'JSH270C-PO'  

start = time.time()
process_type = 'Default'
df_po = pd.read_excel('Test Case Data.xlsx', sheet_name= 'PO')
df_st = pd.read_excel('Test Case Data.xlsx', sheet_name= 'MC') 
df_po['Stock_Name'] = 'HSC'
df_st['Stock_Name'] = 'HSC'
# get_po_data(df_po,Thickness,Customer,Spec,stock_name)
po_data_raw = get_po_data(df_po,Thickness,Spec,stock_name)
stock_data_raw = get_stock_data(df_st,Thickness,Spec)

df = po_data_raw.reset_index(drop=True)
df_stock = stock_data_raw.reset_index(drop=True)

final_output = output_table(df_po,df_st , Thickness,Spec,stock_name)
final_output[['Order_ID', 'Customer', 'Spec_Name', 'Thickness', 'Width','Need_Cut','Inventory_ID', 'Mother_Weight', 'Mother_Width',
        'Max_Weight', 'Expected_Qt', 'Num_Split', 'Max_Quant', 'Total_Quant','Trim_Loss','Trim_Loss_Weight', 'Total_MC', 'Total_Remain','Total_Trim_Loss_Weight']]

Lần cắt thứ 1
So coil con lai:  0

        Total_MC: 1
        Total_Trim_Loss_Weight: 59.581624282198526
        Total_Remain: 2390.4183757178016     
        Remain Order: 0 
        
Lần cắt thứ 1
Trong kho có 1 MC ... Số MC này không đủ để cắt !!!! 
Số Coil còn lại: 6
Lần cắt thứ 1
So coil con lai:  0

        Total_MC: 1
        Total_Trim_Loss_Weight: 59.581624282198526
        Total_Remain: 2390.4183757178016     
        Remain Order: 0 
        
Lần cắt thứ 1
Trong kho có 1 MC ... Số MC này không đủ để cắt !!!! 
Số Coil còn lại: 6
Lần cắt thứ 1
So coil con lai:  0

        Total_MC: 1
        Total_Trim_Loss_Weight: 59.581624282198526
        Total_Remain: 2390.4183757178016     
        Remain Order: 0 
        
Lần cắt thứ 1
Trong kho có 1 MC ... Số MC này không đủ để cắt !!!! 
Số Coil còn lại: 6
Lần cắt thứ 1
So coil con lai:  0

        Total_MC: 1
        Total_Trim_Loss_Weight: 59.581624282198526
        Total_Remain: 2390.4183757178016     
        Remain Order: 0 
    

,Order_ID,Customer,Spec_Name,Thickness,Width,Need_Cut,Inventory_ID,Mother_Weight,Mother_Width,Max_Weight,Expected_Qt,Num_Split,Max_Quant,Total_Quant,Trim_Loss,Trim_Loss_Weight,Total_MC,Total_Remain,Total_Trim_Loss_Weight
0,0,CIC,JSH270C-PO,2.3,110.0,-1240.0,HTV0876/23,8070.0,1219,-216.439705,2,2.0,2,2.0,9.0,59.581624,1,2390.418376,59.581624
1,1,CIC,JSH270C-PO,2.3,145.0,-1097.0,HTV0876/23,8070.0,1219,-822.852338,2,2.0,2,2.0,9.0,59.581624,1,2390.418376,59.581624
2,2,CIC,JSH270C-PO,2.3,50.0,-790.0,HTV0876/23,8070.0,1219,-203.027071,3,3.0,3,3.0,9.0,59.581624,1,2390.418376,59.581624
3,3,CIC,JSH270C-PO,2.3,55.0,-700.0,HTV0876/23,8070.0,1219,-28.219852,2,2.0,2,2.0,9.0,59.581624,1,2390.418376,59.581624
4,4,CIC,JSH270C-PO,2.3,120.0,-668.0,HTV0876/23,8070.0,1219,-126.421657,1,1.0,1,1.0,9.0,59.581624,1,2390.418376,59.581624
5,5,CIC,JSH270C-PO,2.3,51.0,-500.0,HTV0876/23,8070.0,1219,-175.258409,2,2.0,2,2.0,9.0,59.581624,1,2390.418376,59.581624
6,6,CIC,JSH270C-PO,2.3,128.0,-425.0,HTV0876/23,8070.0,1219,-422.383101,1,1.0,1,1.0,9.0,59.581624,1,2390.418376,59.581624
7,7,CIC,JSH270C-PO,2.3,90.0,-200.0,HTV0876/23,8070.0,1219,-395.816243,1,1.0,1,1.0,9.0,59.581624,1,2390.418376,59.581624
